In [ ]:
import csv, json, math

def LonLatToWebMercator(lon,lat):
    if lon != "" and lat != "":        
        lon = float(lon)
        lat = float(lat)
        if lat > 85.05113:
            lat = 85.05113
        if lat < -85.05113:
            lat = -85.05113
        x = (lon + 180.0) * 256.0 / 360.0
        y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    else:
        x = ""
        y = ""
    return [x, y]

In [ ]:
country_geojson = {
    "type": "FeatureCollections",
    "features": []
}
filename = "gapminder.json"
with open(filename) as f:
    gapminder_data = json.load(f)
    for country in gapminder_data['rows']:
        gapminder_geo = country[0]
        gapminder_name = country[1]
        lat = country[2]
        lng = country[3]
        x,y = LonLatToWebMercator(lng, lat)        
        feature = {
            "type": "Feature",
            "geometry": {"type": "Point", "coordinates": [lng, lat]},
            "properties": {"names": [gapminder_name, gapminder_geo], "webmercator": [x,y]}
        }
        country_geojson["features"].append(feature)
with open('gapminder.geojson', 'w') as f:
    json.dump(country_geojson, f, indent=4)
    
# gapminder_data['headers']
# [u'geo', u'name', u'latitude', u'longitude', u'world_4region']
# gapminder_data['rows'][0]
# [u'afg', u'Afghanistan', 33, 66, u'asia']

In [ ]:
filename = "gapminder.geojson"
with open(filename) as f:
    data = json.load(f)
# data['features'][0]
# {u'geometry': {u'coordinates': [66, 33], u'type': u'Point'},
#  u'properties': {u'gapminder_geo': u'afg',
#  u'gapminder_name': u'Afghanistan',
#  u'webmercator': [174.93333333333334, 103.11672116189807]},
#  u'type': u'Feature'}


In [ ]:
data['features'][0]

In [ ]:
country_data = []
filename = "gothos/country_centroids_all.csv"
with open(filename) as csvfile:
    reader = csv.DictReader(csvfile, delimiter="\t")
    for row in reader:
        country_data.append(row)
#country_data[0]
#{'AFFIL': '',
# 'DMS_LAT': '330000',
# 'DMS_LONG': '660000',
# 'DSG': 'PCLI',
# 'FIPS10': 'AF',
# 'FULL_NAME': 'Islamic Republic of Afghanistan',
# 'ISO3136': 'AF',
# 'JOG': 'NI42-09',
# 'LAT': '33',
# 'LONG': '66',
# 'MGRS': '42STB1970055286',
# 'MOD_DATE': '2009-04-10',
# 'SHORT_NAME': 'Afghanistan'}

In [ ]:
i = 0
for feature in data['features']:
    gapminder_name = feature['properties']['names'][0]    
    match = False
    for row in country_data:
        short_name = row['SHORT_NAME']
        if short_name == gapminder_name:
            match = True
            data['features'][i]['properties']['names'].append(row['ISO3136'])             
            break
    if not match:
        print "Did not find %s" % gapminder_name
    i+= 1

In [ ]:
with open('gapminder.geojson', 'w') as f:
    json.dump(data, f, indent=4)


In [ ]:
filename = "ne_50m_admin_0_countries.geo.json"
with open(filename) as f:
    polygon_data = json.load(f)

In [ ]:
polygon_data['features'][1]['properties']

In [ ]:
for feature in polygon_data['features']:
    properties = feature['properties']
    if properties['name'] != properties['name_long'] or \
        properties['name'] != properties['name_sort'] or \
        properties['name_sort'] != properties['name_long']:
        print "MISMATCH %s %s %s" % (properties['name'], properties['name_long'], properties['name_sort'])

In [ ]:
country_geojson = {
    "type": "FeatureCollections",
    "features": []
}
filename = "country_polygons.geojson"
for feature in polygon_data['features']:
    geometry = feature['geometry']    
    properties = feature['properties']
    name_keys = ['abbrev','adm0_a3','admin','formal_en','iso_a2','iso_a3','iso_n3','name','name_long','name_sort']
    names = []
    for val in name_keys:
        if properties[val] is not None:
            names.append(properties[val])
    feature = {
        "type": "Feature",
        "geometry": geometry,
        "properties": { 'names': names}
        }
    country_geojson["features"].append(feature)
with open(filename, 'w') as f:
    json.dump(country_geojson, f, indent=4)


In [ ]:
for feature in data['features']:
    properties = feature['properties']
    names = properties['names']
    match = False
    for country in country_geojson['features']:
        country_names = country['properties']['names']
        for country_name in country_names:
            for name in names:
                if name == country_name:
                    match = True
                if match:
                    break
        if match:
            names += country_names
#            print "MATCH FOUND: " + names[0]
#            print "             " + ",".join(country_names)
            break

        

In [ ]:
with open('gapminder.geojson', 'w') as f:
    json.dump(data, f, indent=4)
